# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [2]:
from sklearn import ensemble , cross_validation, learning_curve, metrics, datasets 

import numpy as np
import pandas as pd
import xgboost as xgb

dataset = datasets.load_boston()
print(dataset.data[0:5])
print(dataset.data.shape)

# 25% в тест именно последних!
X_train = dataset.data[:379]
X_test = dataset.data[379:]
y_train = dataset.target[:379]
y_test = dataset.target[379:]

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

/home/loki/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/loki/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


[[  6.32000000e-03   1.80000000e+01   2.31000000e+00   0.00000000e+00
    5.38000000e-01   6.57500000e+00   6.52000000e+01   4.09000000e+00
    1.00000000e+00   2.96000000e+02   1.53000000e+01   3.96900000e+02
    4.98000000e+00]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00   0.00000000e+00
    4.69000000e-01   6.42100000e+00   7.89000000e+01   4.96710000e+00
    2.00000000e+00   2.42000000e+02   1.78000000e+01   3.96900000e+02
    9.14000000e+00]
 [  2.72900000e-02   0.00000000e+00   7.07000000e+00   0.00000000e+00
    4.69000000e-01   7.18500000e+00   6.11000000e+01   4.96710000e+00
    2.00000000e+00   2.42000000e+02   1.78000000e+01   3.92830000e+02
    4.03000000e+00]
 [  3.23700000e-02   0.00000000e+00   2.18000000e+00   0.00000000e+00
    4.58000000e-01   6.99800000e+00   4.58000000e+01   6.06220000e+00
    3.00000000e+00   2.22000000e+02   1.87000000e+01   3.94630000e+02
    2.94000000e+00]
 [  6.90500000e-02   0.00000000e+00   2.18000000e+00   0.00000000e+00
    4.5800

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [3]:
def s_vect(Y, Z):
    return Y - Z

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

def gbm_predict(X, base_algorithms_list, coefficients_list):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

base_algorithms_list = []
coefficients_list = []

next_y = y_train

for i in range(50):
    regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
    regressor.fit(X_train, next_y)
    base_algorithms_list.append(regressor)
    coefficients_list.append(0.9)
    next_y = s_vect(y_train, gbm_predict(X_train, base_algorithms_list, coefficients_list))
    print(mean_squared_error(y_train, gbm_predict(X_train, base_algorithms_list, coefficients_list))**0.5)

print(mean_squared_error(y_test, gbm_predict(X_test, base_algorithms_list, coefficients_list))**0.5)

3.51163857154
1.77246618749
1.4574139766
1.18506506214
1.02223054327
0.819647879735
0.706767695283
0.593449279065
0.537579977364
0.445471946216
0.382067006289
0.332347399849
0.297877448253
0.269667243847
0.236993393524
0.206523809647
0.16890467223
0.143490362248
0.130273425877
0.105026128281
0.0918357154587
0.0817000197769
0.0780024083531
0.0668903930729
0.0598052915971
0.049877338057
0.0438263210321
0.0403197552186
0.0395170457578
0.0343772342478
0.0283918394727
0.0257392171096
0.024095036434
0.0220126142465
0.0189508553133
0.0155669203789
0.0145068457422
0.0122546418911
0.0105085386446
0.00976746205746
0.00810747371364
0.00718650816698
0.00605391266342
0.00554173137165
0.00460736648501
0.00370733269336
0.00339727473533
0.00317504082371
0.00270034841871
0.00258447429932
5.47665097417


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [5]:
base_algorithms_list = []
coefficients_list = []

next_y = y_train

for i in range(50):
    regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
    regressor.fit(X_train, next_y)
    base_algorithms_list.append(regressor)
    coefficients_list.append(0.9/(1.0 + i))
    next_y = s_vect(y_train, gbm_predict(X_train, base_algorithms_list, coefficients_list))

print(mean_squared_error(y_test, gbm_predict(X_test, base_algorithms_list, coefficients_list))**0.5)

4.81089328026


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [14]:
for i in range(50, 300):
    regressor = xgb.XGBRegressor(n_estimators = i)
    regressor.fit(X_train, y_train)
    predictions = regressor.predict(X_train)
    print(mean_squared_error(y_train, predictions))
    predictions_test = regressor.predict(X_test)
    print(mean_squared_error(y_test, predictions_test))
    print("----------------------------------")

3.12024668119
26.368976036
----------------------------------
3.07208039855
26.1578626838
----------------------------------
3.00135066389
26.0991119464
----------------------------------
2.95497111992
26.1231935334
----------------------------------
2.90296403111
26.1726595361
----------------------------------
2.86696040741
25.7171338719
----------------------------------
2.83691077951
25.7203911738
----------------------------------
2.78951368508
25.9965600772
----------------------------------
2.74700297636
25.9586224916
----------------------------------
2.71933168432
25.5562358268
----------------------------------
2.65958540966
25.47360316
----------------------------------
2.61396093851
25.4691335861
----------------------------------
2.58011158034
25.5167887042
----------------------------------
2.53430011747
25.3831788507
----------------------------------
2.50453440528
25.3887702933
----------------------------------
2.48096497921
25.1435507448
------------------------------

0.892664060945
21.7787243506
----------------------------------
0.887204675537
21.7734370285
----------------------------------
0.883040910039
21.7347167564
----------------------------------
0.876554186069
21.7429863233
----------------------------------
0.865906617622
21.7333113342
----------------------------------
0.857076071419
21.7094193029
----------------------------------
0.843758828111
21.6886881403
----------------------------------
0.830032060395
21.664739945
----------------------------------
0.824230787809
21.6211297131
----------------------------------
0.817031116263
21.6078157243
----------------------------------
0.814366689232
21.6085008471
----------------------------------
0.810792312686
21.599458443
----------------------------------
0.805543347597
21.634622858
----------------------------------
0.798103927474
21.6496676676
----------------------------------
0.78596754712
21.6032114772
----------------------------------
0.776524112157
21.63411045
-----------------

In [13]:
for i in range(2, 12):
    regressor = xgb.XGBRegressor(max_depth = i)
    regressor.fit(X_train, y_train)
    predictions = regressor.predict(X_train)
    print(mean_squared_error(y_train, predictions))
    predictions_test = regressor.predict(X_test)
    print(mean_squared_error(y_test, predictions_test))
    print("----------------------------------")

3.90729135843
26.9670876623
----------------------------------
1.69953226516
22.8596700183
----------------------------------
0.744624905949
24.2884357358
----------------------------------
0.342737999194
23.8422662007
----------------------------------
0.106319424247
24.0085909942
----------------------------------
0.041087840513
28.1290795305
----------------------------------
0.0185673555587
26.5086508027
----------------------------------
0.0089921023046
26.6564051534
----------------------------------
0.00352158894552
27.5258312101
----------------------------------
0.00142750998706
25.2080932259
----------------------------------


## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [19]:
from  sklearn import linear_model

regression = linear_model.LinearRegression()
regression.fit(X_train, y_train)
predictions_test = regression.predict(X_test)
print(mean_squared_error(y_test, predictions_test)**0.5)

8.27046803494
